## Exercise 2

*a) Generate a simulated data set as follows:*

In [14]:
srand(1)
x = randn(100) 
y = x - 2x.^2 + randn(100)
;

*In this data set, what is n and what is p? Write out the model used to generate the data in equation form*

In this regression model $n = 100$ which is the length of the variables $x$ and $y$, and $p=2$, since $y$ is generated with two predictos $x$ and $x^2$, plus a  random error.

*(c) Do 5-fold cross-validation to fit polynomial models using least squares, we compute the generalization error*

Step 1: Create the folds

In [15]:
n = 100
k = 5
s = convert(Integer, n / k)
seq = randperm(n)
fold = [seq[((i-1)*s + 1):(i*s)] for i in 1:k]

5-element Array{Array{Int64,1},1}:
 [45, 92, 34, 86, 76, 30, 94, 88, 97, 79, 66, 100, 21, 80, 4, 72, 59, 42, 69, 31]
 [14, 85, 5, 19, 74, 6, 48, 89, 60, 75, 87, 83, 81, 15, 71, 23, 27, 40, 41, 29]  
 [82, 49, 22, 44, 17, 9, 73, 13, 28, 56, 11, 1, 7, 36, 20, 33, 47, 35, 68, 63]   
 [84, 95, 99, 98, 77, 61, 24, 38, 53, 16, 62, 52, 43, 3, 54, 91, 25, 39, 26, 32] 
 [65, 93, 2, 70, 67, 46, 64, 90, 57, 12, 78, 10, 37, 18, 51, 55, 96, 8, 50, 58]  

Step 2: function for lm fitting

In [39]:
function fitreg(deg, x, y)
    # julia is preoptimized for descent solutions methods w/short code
    X = ones(length(y))
    for i in 1:deg
       X = [X x.^i] 
    end
    βhat = Symmetric(X'*X) \ X'*y
    return βhat, X  
end

polyreg (generic function with 1 method)

Step 3: fit models with cross-validation

In [60]:
# design matrix with polynomial entries
X = [ones(length(y)) x x.^2 x.^3 x.^4]   
# store espace for generalization error
gen_error = Array{Float64}(k)
# k-fold cross validation
for i in 1:k # outer loop is the cross-validation loop
    # test and train indices
    train_idx = fold[i]
    test_idx = vcat([fold[j] for j in 1:k if j != 3]...)
    # y data
    y_train = y[train_idx]
    y_test = y[test_idx]
    # fit model
    for deg in 1:4 # inner loop fits different polynomial models
        # X data
        X_train = X[train_idx, 1:(1 + deg)]
        X_test = X[test_idx, 1:(1 + deg)]
        # regression
        βhat_train = Symmetric(X_train'*X_train) \ X_train'*y_train
        yhat_test = X_test*βhat_train
        gen_error[deg] += mean((yhat_test - y_test).^2) / k # average cross validation error        
    end
end

In [65]:

@printf("degree: %i, mse: %f", 1:4, gen_error)

degree: 

Stacktrace:
 [1] depwarn(::String, ::Symbol) at .\deprecated.jl:70
 [2] isfinite(::UnitRange{Int64}) at .\deprecated.jl:57
 [3] include_string(::String, ::String) at .\loading.jl:515
 [4] include_string(::Module, ::String, ::String) at C:\Users\mbtec\.julia\v0.6\Compat\src\Compat.jl:407
 [5] execute_request(::ZMQ.Socket, ::IJulia.Msg) at C:\Users\mbtec\.julia\v0.6\IJulia\src\execute_request.jl:154
 [6] eventloop(::ZMQ.Socket) at C:\Users\mbtec\.julia\v0.6\IJulia\src\eventloop.jl:8
 [7] (::IJulia.##14#17)() at .\task.jl:335
while loading In[65], in expression starting on line 1


LoadError: [91mTypeError: non-boolean (BitArray{1}) used in boolean context[39m

In [56]:
using Plots

In [57]:
plot(gen_error, seriestype = :bar, xlab = "degree", ylab = "mse", 
    title = "mean square error by polynomial degree", legend = false)

5-element Array{Float64,1}:
 8.77228     
 1.11765     
 2.53494     
 2.16768     
 6.55815e-316